In [1]:
import torch

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

data_train = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
data_test = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

In [3]:
from torch.utils.data import DataLoader

batch_size = 64
# wraps an iterable over dataset, supports automatic batching, sampling, shuffling and multiprocess data loading
dl_train = DataLoader(data_train, batch_size=batch_size)
dl_test = DataLoader(data_test, batch_size=batch_size)

In [4]:
for x, y in dl_test:
    print(f"Shape of x [batch_size, channel, height, weight]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of x [batch_size, channel, height, weight]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
_, h, w = data_test.data.shape
in_features = h*w
out_features = 10
in_features, out_features

(784, 10)

In [6]:
from torch import nn

class NNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_features)
        )

    def forward(self, x):
        x = self.flatten(x)
        out = self.linear_relu_stack(x)
        return out

In [7]:
model = NNModel().to(device)
print(model)

NNModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dl, model, loss_fn, optimizer):
    size = len(dl.dataset)
    model.train()

    for i, (x, y) in enumerate(dl):
        x, y = x.to(device), y.to(device)
        
        logits = model(x)
        loss = loss_fn(logits, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            l, c = loss.item(), len(x)*(i + 1)
            print(f"loss: {l:>7f} [{c:>5d}/{size:>5d}]")

In [10]:
def test(dl, model, loss_fn):
    size = len(dl.dataset)
    model.eval()

    loss, hits = 0, 0
    with torch.no_grad():
        for x, y in dl:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss += loss_fn(logits, y).item()
            hits += (logits.argmax(1) == y).type(torch.float).sum().item()
    avg_loss = loss / len(dl)
    accu = hits / size
    print(f"Test: \n Accuracy: {(100*accu):>0.1f}%, Average loss: {avg_loss:>8f} \n")

In [11]:
epochs = 5
for i in range(epochs):
    print(f"------------- Epoch {i+1} -------------\n")
    train(dl_train, model, loss_fn, optimizer)
    test(dl_test, model, loss_fn)
print("Complete")

------------- Epoch 1 -------------

loss: 2.294734 [   64/60000]
loss: 2.277577 [ 6464/60000]
loss: 2.263461 [12864/60000]
loss: 2.265402 [19264/60000]
loss: 2.248147 [25664/60000]
loss: 2.217117 [32064/60000]
loss: 2.221019 [38464/60000]
loss: 2.183275 [44864/60000]
loss: 2.185709 [51264/60000]
loss: 2.157821 [57664/60000]
Test: 
 Accuracy: 50.6%, Average loss: 2.146639 

------------- Epoch 2 -------------

loss: 2.158524 [   64/60000]
loss: 2.142937 [ 6464/60000]
loss: 2.084475 [12864/60000]
loss: 2.112900 [19264/60000]
loss: 2.060873 [25664/60000]
loss: 1.999957 [32064/60000]
loss: 2.026592 [38464/60000]
loss: 1.938414 [44864/60000]
loss: 1.953390 [51264/60000]
loss: 1.886112 [57664/60000]
Test: 
 Accuracy: 54.8%, Average loss: 1.873158 

------------- Epoch 3 -------------

loss: 1.913509 [   64/60000]
loss: 1.873739 [ 6464/60000]
loss: 1.752289 [12864/60000]
loss: 1.808625 [19264/60000]
loss: 1.699223 [25664/60000]
loss: 1.648998 [32064/60000]
loss: 1.675746 [38464/60000]
loss: 

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Model saved.")

Model saved.
